In [1]:
import pandas as pd
import csv
import os

import linecache
import sys

import matplotlib.pyplot as plt
import math
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import plotly.graph_objs as go

#from prettytable import PrettyTable

#簡轉繁用 
#https://pypi.python.org/pypi/hanziconv/0.2.1
from hanziconv import HanziConv

import matplotlib_venn 
from matplotlib_venn import venn2, venn2_circles

import collections
from collections import Counter
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error
from numpy import mean, sqrt, square, arange

import pickle

import random
import datetime
import operator

#https://stackoverflow.com/questions/14519177/python-exception-handling-line-number
def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print ('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))


In [2]:
import multiprocessing

def _apply(pool_map_args):
    df = pool_map_args[0]
    func = pool_map_args[1]
    args = pool_map_args[2]
#     print(args)
    
    return df.apply(func,axis = 1, args = args)

def parallel_apply(df, func, args):
    #指定主機有多少CPU，並把df 平均分配size
    num_processes = 32
    chunk_size = int(df.shape[0]/num_processes)

    #將df 拆解成 num_processes 個 chunk 的df list
    chunks = [df.ix[df.index[i:i + chunk_size]] for i in range(0, df.shape[0], chunk_size)]
    pool_map_args = [(df, func, args) for df in chunks]
    pool = multiprocessing.Pool(processes = num_processes)
    result = pool.map(_apply, pool_map_args)
    result = pd.concat([i for i in result])
    pool.terminate()
    pool.join()
    return result

In [3]:
with open('./pkl_data/special_word_dict.pkl', 'rb') as handle:
    special_word_dict = pickle.loads(handle.read())
    
with open('./pkl_data/moe_additional_dict.pkl', 'rb') as handle:
    moe_additional_dict = pickle.loads(handle.read())

with open('./pkl_data/Totalname_list.pkl', 'rb') as handle:
    Totalname_list = pickle.loads(handle.read())
    
with open('./pkl_data/son_in_list.pkl', 'rb') as handle:
    son_in_list = pickle.loads(handle.read())
    
with open('./pkl_data/mu_in_list.pkl', 'rb') as handle:
    mu_in_list = pickle.loads(handle.read())

In [4]:
# READ MY DATA INTO CSV FILE
msadata = pd.read_csv('./NameData/EnglishReconcateRealTaiwan.csv', parse_dates = True)

In [5]:
# READ RECONCATE DATA
reconcatedata = pd.read_csv('./NameData/EnglishReconcateRealTaiwan.csv', parse_dates = True)

/home/yvonnechow/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning:

Columns (220) have mixed types. Specify dtype option on import or set low_memory=False.



In [6]:
age_range_list = msadata['BirthYear'].tolist()

# RFC

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

def dataset_statistics(dataset):
    """
    Basic statistics of the dataset
    :param dataset: Pandas dataframe
    :return: None, print the basic statistics of the dataset
    """
    print (dataset.describe())
    
#http://dataaspirant.com/2017/06/26/random-forest-classifier-python-scikit-learn/
#Split train and test
def split_dataset(dataset, train_percentage, feature_headers, target_header):
    """
    Split the dataset with train_percentage
    :param dataset:
    :param train_percentage:
    :param feature_headers:
    :param target_header:
    :return: train_x, test_x, train_y, test_y
    """

    # Split dataset into train and test dataset
    train_x, test_x, train_y, test_y = train_test_split(dataset[feature_headers], dataset[target_header],
                                                        train_size = train_percentage)
    return train_x, test_x, train_y, test_y

def random_forest_classifier(features, target):
    """
    To train the random forest classifier with features and target data
    :param features:
    :param target:
    :return: trained random forest classifier
    """
    clf = RandomForestClassifier(n_estimators = 101, n_jobs = -1 , min_samples_leaf = 1)
    clf.fit(features, target)
    return clf

In [8]:
def basic_RFClassifier(data_df,x_feature,y_feature,Validation_times):

    #N fold cross-validation
    Train_accuracy_score = 0
    Test_accuracy_score = 0
    
    for i in range(Validation_times):
        train_x, test_x, train_y, test_y = split_dataset(data_df, 0.7, x_feature, y_feature)
        train_index = train_x.index
        test_index = test_x.index
 
        # Create random forest classifier instance
        trained_model = random_forest_classifier(train_x, train_y)
        
        predictions = trained_model.predict(test_x)
#         print(predictions)

        # Train and Test Accuracy
        if(Validation_times!=1):
            print('Run times:', i+1)
        print ("Train Accuracy :: ", accuracy_score(train_y, trained_model.predict(train_x)))
        print ("Test Accuracy  :: ", accuracy_score(test_y, predictions))
        #print ("Confusion matrix\n ", confusion_matrix(test_y, predictions))

        Train_accuracy_score += accuracy_score(train_y, trained_model.predict(train_x))
        Test_accuracy_score += accuracy_score(test_y, predictions)
        #calculate_error(predictions, test_y )
        #validate_error(predictions, test_y, test_x , data_df)
        
    if(Validation_times != 1):
        Train_accuracy_score /= Validation_times
        Test_accuracy_score /= Validation_times
        print ("Average Train Accuracy :: ", Train_accuracy_score)
        print ("Average Test Accuracy  :: ", Test_accuracy_score)
    
    return trained_model

In [9]:
# # READ TRAIN TEST INDEX
with open('./NameData/train_test_data/Englishtrain_index', 'rb') as handle:
    train_index = pickle.loads(handle.read())
    
with open('./NameData/train_test_data/Englishtest_index', 'rb') as handle:
    test_index = pickle.loads(handle.read())

In [10]:
# y_feature = 'BirthYear'

test_x = msadata.copy()
# test_y = msadata.y_feature
test_index = test_x.index
print(len(test_index))

%xdel test_x
# %xdel test_y

30565


# Save model

In [11]:
# FN1

x_feature = ['FN1']
y_feature = ['BirthYear']
    
age_test = basic_RFClassifier(reconcatedata, x_feature, y_feature, 10)

# Save the model to disk
filename = './concate_age_model/age_predicton_FN1.sav'
pickle.dump(age_test, open(filename, 'wb'))
print("Model save successfully!")

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.20618465158612118
Test Accuracy  ::  0.18004765357919816


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.20565631451595
Test Accuracy  ::  0.1813633067440174


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.2053810464625835
Test Accuracy  ::  0.1815497772713146


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.20526117166514973
Test Accuracy  ::  0.18036879726509894


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.2050968987945923
Test Accuracy  ::  0.18337304464933182
Average Train Accuracy ::  0.20545918707128114
Average Test Accuracy  ::  0.18131254532269758
Model save successfully!


In [12]:
# FN2

x_feature = ['FN2']
y_feature = ['BirthYear']
    
age_test = basic_RFClassifier(reconcatedata, x_feature, y_feature, 10)

# Save the model to disk
filename = './concate_age_model/age_predicton_FN2.sav'
pickle.dump(age_test, open(filename, 'wb'))
print("Model save successfully!")

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.21486003507447776
Test Accuracy  ::  0.18743395835491558


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.21505982640353408
Test Accuracy  ::  0.1865637625608619


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.2144515728017404
Test Accuracy  ::  0.18699886045788874


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.2154860479055209
Test Accuracy  ::  0.18607686729514142


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.21468688258929564
Test Accuracy  ::  0.18755827203978037
Average Train Accuracy ::  0.2148382800186472
Average Test Accuracy  ::  0.18706412514244278
Model save successfully!


In [13]:
# 2F

x_feature = ['FN1','FN2']
y_feature = ['BirthYear']
    
age_test = basic_RFClassifier(reconcatedata, x_feature, y_feature, 10)

# Save the model to disk
filename = './concate_age_model/age_predicton_FN12.sav'
pickle.dump(age_test, open(filename, 'wb'))
print("Model save successfully!")

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.556978267143206
Test Accuracy  ::  0.2006940847404952


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.5556552045641219
Test Accuracy  ::  0.19937843157567595


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.5561968610562301
Test Accuracy  ::  0.19933699368072103


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.5572224565453859
Test Accuracy  ::  0.19838392209675748


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.5567473971629632
Test Accuracy  ::  0.19870506578265823
Average Train Accuracy ::  0.5568770395364842
Average Test Accuracy  ::  0.19869574225629338
Model save successfully!


In [14]:
# sonin

x_feature = ['FN1_sonin_index','FN2_sonin_index']
y_feature = ['BirthYear']
    
age_test = basic_RFClassifier(reconcatedata, x_feature, y_feature, 10)

# Save the model to disk
filename = './concate_age_model/age_predicton_sonin.sav'
pickle.dump(age_test, open(filename, 'wb'))
print("Model save successfully!")

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.1657602060070593
Test Accuracy  ::  0.14692841603646534


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.16515195240526562
Test Accuracy  ::  0.14867916709831142


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.1655248962195041
Test Accuracy  ::  0.1479850823578162


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.16485004550802496
Test Accuracy  ::  0.14873096446700507


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.16517415144182743
Test Accuracy  ::  0.14860665078214028
Average Train Accuracy ::  0.1653659511177215
Average Test Accuracy  ::  0.14776235367243343
Model save successfully!


In [15]:
# muin

x_feature = ['FN1_muin_index','FN2_muin_index']
y_feature = ['BirthYear']
    
age_test = basic_RFClassifier(reconcatedata, x_feature, y_feature, 10)

# Save the model to disk
filename = './concate_age_model/age_predicton_muin.sav'
pickle.dump(age_test, open(filename, 'wb'))
print("Model save successfully!")

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.15528226074988347
Test Accuracy  ::  0.13874443178286544


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.15542877439119143
Test Accuracy  ::  0.13894126178390137


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.15572624148111971
Test Accuracy  ::  0.13839220967574847


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.1555619686105623
Test Accuracy  ::  0.13748057598673988


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.1555486491886252
Test Accuracy  ::  0.1382471770434062
Average Train Accuracy ::  0.15551712655670746
Average Test Accuracy  ::  0.13795918367346938
Model save successfully!


In [16]:
# FN1+sonin1

x_feature = ['FN1','FN1_sonin_index']
y_feature = ['BirthYear']
    
age_test = basic_RFClassifier(reconcatedata, x_feature, y_feature, 10)

# Save the model to disk
filename = './concate_age_model/age_predicton_FN1sonin1.sav'
pickle.dump(age_test, open(filename, 'wb'))
print("Model save successfully!")

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.2054565231868937
Test Accuracy  ::  0.182274940433026


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.20522121339933846
Test Accuracy  ::  0.1804516730550088


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.20500810264834507
Test Accuracy  ::  0.18190199937843157


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.20526561147246208
Test Accuracy  ::  0.18096964674194552


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.20580726796457033
Test Accuracy  ::  0.18064850305604474
Average Train Accuracy ::  0.20518924678668946
Average Test Accuracy  ::  0.18183466279912983
Model save successfully!


In [17]:
# FN1+muin1

x_feature = ['FN1','FN1_muin_index']
y_feature = ['BirthYear']
    
age_test = basic_RFClassifier(reconcatedata, x_feature, y_feature, 10)

# Save the model to disk
filename = './concate_age_model/age_predicton_FN1muin1.sav'
pickle.dump(age_test, open(filename, 'wb'))
print("Model save successfully!")

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.2056918329744489
Test Accuracy  ::  0.18074173831969337


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.20511465802384177
Test Accuracy  ::  0.18159121516626955


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.20485714919972473
Test Accuracy  ::  0.1820677509582513


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.20587830488156814
Test Accuracy  ::  0.18113539832176526


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.2054920416453926
Test Accuracy  ::  0.1810525225318554
Average Train Accuracy ::  0.2053113414877794
Average Test Accuracy  ::  0.1816430125349632
Model save successfully!


In [18]:
# FN2+sonin2

x_feature = ['FN2','FN2_sonin_index']
y_feature = ['BirthYear']
    
age_test = basic_RFClassifier(reconcatedata, x_feature, y_feature, 10)

# Save the model to disk
filename = './concate_age_model/age_predicton_FN2sonin2.sav'
pickle.dump(age_test, open(filename, 'wb'))
print("Model save successfully!")

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.2151175438985948
Test Accuracy  ::  0.18676059256189786


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.21444713299442805
Test Accuracy  ::  0.1885631409924376


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.21418518436299866
Test Accuracy  ::  0.18836631099140164


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.21582347326126047
Test Accuracy  ::  0.18489588728892573


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.2148555952671654
Test Accuracy  ::  0.18608722676888015
Average Train Accuracy ::  0.21471041356805118
Average Test Accuracy  ::  0.18697296177354192
Model save successfully!


In [19]:
# FN2+muin2

x_feature = ['FN2','FN2_muin_index']
y_feature = ['BirthYear']
    
age_test = basic_RFClassifier(reconcatedata, x_feature, y_feature, 10)

# Save the model to disk
filename = './concate_age_model/age_predicton_FN2muin2.sav'
pickle.dump(age_test, open(filename, 'wb'))
print("Model save successfully!")

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.21505538659622173
Test Accuracy  ::  0.18705065782658242


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.21477123892823052
Test Accuracy  ::  0.18761006940847405


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.21539725175927366
Test Accuracy  ::  0.18624261887496116


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.21462472528692256
Test Accuracy  ::  0.18734072309126695


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.2143450174262437
Test Accuracy  ::  0.18863565730860873
Average Train Accuracy ::  0.21493906364463786
Average Test Accuracy  ::  0.1868942297731275
Model save successfully!


In [20]:
# FN+sonin

x_feature = ['FN1','FN2','FN1_sonin_index','FN2_sonin_index']
y_feature = ['BirthYear']
    
age_test = basic_RFClassifier(reconcatedata, x_feature, y_feature, 10)

# Save the model to disk
filename = './concate_age_model/age_predicton_FNsonin.sav'
pickle.dump(age_test, open(filename, 'wb'))
print("Model save successfully!")

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.5561791018269807
Test Accuracy  ::  0.1997824510514866


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.5565076475680956
Test Accuracy  ::  0.2008805552677924


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.5571070215552645
Test Accuracy  ::  0.19870506578265823


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.556880591382334
Test Accuracy  ::  0.2005801305293691


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.5569383088773947
Test Accuracy  ::  0.19961669947166685
Average Train Accuracy ::  0.5569485204342131
Average Test Accuracy  ::  0.2000310784212162
Model save successfully!


In [21]:
# FN+muin

x_feature = ['FN1','FN2','FN1_muin_index','FN2_muin_index']
y_feature = ['BirthYear']
    
age_test = basic_RFClassifier(reconcatedata, x_feature, y_feature, 10)

# Save the model to disk
filename = './concate_age_model/age_predicton_FNmuin.sav'
pickle.dump(age_test, open(filename, 'wb'))
print("Model save successfully!")

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.5561080649099829
Test Accuracy  ::  0.20110846369004454


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.5562013008635425
Test Accuracy  ::  0.19938879104941468


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.5558638755078029
Test Accuracy  ::  0.2007355226354501


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.5572979332696961
Test Accuracy  ::  0.1990572878897752


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.5568228738872733
Test Accuracy  ::  0.2000310784212162
Average Train Accuracy ::  0.5567806957178059
Average Test Accuracy  ::  0.19977312752512172
Model save successfully!


In [22]:
# sonin+muin

x_feature = ['FN1_sonin_index','FN2_sonin_index','FN1_muin_index','FN2_muin_index']
y_feature = ['BirthYear']
    
age_test = basic_RFClassifier(reconcatedata, x_feature, y_feature, 10)

# Save the model to disk
filename = './concate_age_model/age_predicton_soninmuin.sav'
pickle.dump(age_test, open(filename, 'wb'))
print("Model save successfully!")

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.32660554531933317
Test Accuracy  ::  0.18190199937843157


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.3273691921770595
Test Accuracy  ::  0.18209882937946753


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.3267476191533287
Test Accuracy  ::  0.18192271832590903


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.32679201722645235
Test Accuracy  ::  0.18176732621982802


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.32637467533909026
Test Accuracy  ::  0.18231637832798095
Average Train Accuracy ::  0.3269802650564966
Average Test Accuracy  ::  0.18215787837977832
Model save successfully!


In [33]:
# 6F

x_feature = ['FN1','FN2','FN1_sonin_index','FN2_sonin_index','FN1_muin_index','FN2_muin_index']
y_feature = ['BirthYear']
    
age_test = basic_RFClassifier(reconcatedata, x_feature, y_feature, 10)

# Save the model to disk
filename = './concate_age_model/age_predicton_6F.sav'
pickle.dump(age_test, open(filename, 'wb'))
print("Model save successfully!")

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.555926032810176
Test Accuracy  ::  0.2031907179115301


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.5569027904188958
Test Accuracy  ::  0.2023723194861701


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.5568450729238351
Test Accuracy  ::  0.20025898684346835


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.5568317535018981
Test Accuracy  ::  0.2026209468558997


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.5562945368171022
Test Accuracy  ::  0.202765979488242
Average Train Accuracy ::  0.5565875640997181
Average Test Accuracy  ::  0.20238475085465657
Model save successfully!


In [34]:
# Only FN1 features

x_feature = ['FN1','FN1_sonin_index','FN1_muin_index']
y_feature = ['BirthYear']
    
age_test = basic_RFClassifier(reconcatedata, x_feature, y_feature, 10)

# Save the model to disk
filename = './concate_age_model/age_predicton_FN1Features.sav'
pickle.dump(age_test, open(filename, 'wb'))
print("Model save successfully!")

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.20541212511377005
Test Accuracy  ::  0.18019268621154044


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.2051590560969654
Test Accuracy  ::  0.18198487516834144


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.20501254245565742
Test Accuracy  ::  0.18170516937739561


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.2054432037649566
Test Accuracy  ::  0.1808764114782969


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.2047683530534775
Test Accuracy  ::  0.18201595358955766
Average Train Accuracy ::  0.20529580216218618
Average Test Accuracy  ::  0.18142028384958042
Model save successfully!


In [35]:
# Only FN2 features

x_feature = ['FN2','FN2_sonin_index','FN2_muin_index']
y_feature = ['BirthYear']
    
age_test = basic_RFClassifier(reconcatedata, x_feature, y_feature, 10)

# Save the model to disk
filename = './concate_age_model/age_predicton_FN2Features.sav'
pickle.dump(age_test, open(filename, 'wb'))
print("Model save successfully!")

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.2147490398916687
Test Accuracy  ::  0.18636693255982595


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.2150109885230981
Test Accuracy  ::  0.18687454677302393


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.21453148933336294
Test Accuracy  ::  0.1879519320418523


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.2144737718383022
Test Accuracy  ::  0.18868745467730239


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.2144782116456146
Test Accuracy  ::  0.1890811146793743
Average Train Accuracy ::  0.21476591115945567
Average Test Accuracy  ::  0.18751579819745157
Model save successfully!


# Read model

In [36]:
# Read model file

filename = './concate_age_model/age_predicton_FN1.sav'
age_prediction_FN1 = pickle.load(open(filename, 'rb'))

filename = './concate_age_model/age_predicton_FN2.sav'
age_prediction_FN2 = pickle.load(open(filename, 'rb'))

filename = './concate_age_model/age_predicton_FN12.sav'
age_prediction_FN12 = pickle.load(open(filename, 'rb'))

filename = './concate_age_model/age_predicton_muin.sav'
age_prediction_muin = pickle.load(open(filename, 'rb'))

filename = './concate_age_model/age_predicton_sonin.sav'
age_prediction_sonin = pickle.load(open(filename, 'rb'))

filename = './concate_age_model/age_predicton_soninmuin.sav'
age_prediction_soninmuin = pickle.load(open(filename, 'rb'))

filename = './concate_age_model/age_predicton_FNsonin.sav'
age_prediction_FNsonin = pickle.load(open(filename, 'rb'))

filename = './concate_age_model/age_predicton_FNmuin.sav'
age_prediction_FNmuin = pickle.load(open(filename, 'rb'))

filename = './concate_age_model/age_predicton_6F.sav'
age_prediction_6F = pickle.load(open(filename, 'rb'))

filename = './concate_age_model/age_predicton_FN1Features.sav'
age_predicton_FN1Features = pickle.load(open(filename, 'rb'))

filename = './concate_age_model/age_predicton_FN2Features.sav'
age_predicton_FN2Features = pickle.load(open(filename, 'rb'))

filename = './concate_age_model/age_predicton_FN1sonin1.sav'
age_predicton_FN1sonin1 = pickle.load(open(filename, 'rb'))

filename = './concate_age_model/age_predicton_FN1muin1.sav'
age_predicton_FN1muin1 = pickle.load(open(filename, 'rb'))

filename = './concate_age_model/age_predicton_FN2sonin2.sav'
age_predicton_FN2sonin2 = pickle.load(open(filename, 'rb'))

filename = './concate_age_model/age_predicton_FN2muin2.sav'
age_predicton_FN2muin2 = pickle.load(open(filename, 'rb'))

# Check name frequency

In [11]:
import datetime
import random
def get_test_name_dict_list(test_index,data_df,year_column):
    test_name_dict_list = {}
    FN_list = data_df.loc[test_index]['FirstName'].tolist()
    #year - message / birthyear
    Year_list = data_df.loc[test_index][year_column].tolist()
    for index in range(len(FN_list)):
        FN = FN_list[index]
        year = Year_list[index]
        #print(year)
        if FN in test_name_dict_list:
            if year not in test_name_dict_list[FN]:
                test_name_dict_list[FN][year] = 1
            else:
                test_name_dict_list[FN][year] += 1
        else:
            test_name_dict_list[FN] = {}
            test_name_dict_list[FN][year] = 1
            
    return test_name_dict_list

def get_min_distance(a,b_list):
    smallest_dist = 999
    a = 1940 + a * 5
    
    min_error_answer = 0
    for number in b_list:
        if int(number) > a:
            year = a + 4
        else:
            year = a
        dist = abs(year - int(number))
        if(dist < smallest_dist):
            smallest_dist = dist
    #print(a,b_list,smallest_dist)
    return smallest_dist

In [12]:
def count_average_error(data_df, predictions):
    test_name_list = data_df.loc[test_index].FirstName.tolist()
    # print(len(test_name_list))
    old_score = 0
    new_score = 0
    always_score = 0
    old_count = 0
    new_count = 0
    always_count = 0
    not_match = []

    dist_error = 0
    accuracy = 0
    year_error = [0 for x in range(6,13)]
    year_count = [0 for x in range(6,13)]

    for test_no,year in enumerate(predictions):
        name = test_name_list[test_no]
        if name in test_name_dict_list:
            year_count[len(test_name_dict_list[name])-1] += 1

            if year in test_name_dict_list[name]:
                accuracy += 1
            else:
                #計算錯誤，把平均絕對值誤差加上去
                error = get_min_distance(year,test_name_year_dict_list[name]) 
                dist_error += error

                year_error[len(test_name_dict_list[name])-1] += error

                if name in old_name_list:
                    old_score += error
                    old_count += 1
                elif name in new_name_list:
                    new_score += error
                    new_count += 1

                elif name in always_name_list:
                    always_score += error
                    always_count += 1
        else:
            print('arghhh test name not in test_name_dict_list')

    accuracy/=len(predictions)      
    
    print('Total test list:', len(predictions))
    print('Accuracy:', accuracy)
    print('Mean Year Absolute Error:', dist_error/len(predictions))

In [13]:
test_name_dict_list = get_test_name_dict_list(test_index,msadata,'BirthYear')
test_name_year_dict_list = get_test_name_dict_list(test_index,msadata,'message')
print('test dataset 有',len(msadata),'個名字,',len(test_name_dict_list),'種名字')

test dataset 有 30565 個名字, 19449 種名字


In [14]:
new_name_list = []
always_name_list = []
old_name_list = []
#1~5是old 6~10是new 兩者都有是always
for name in test_name_dict_list:
    new_flag = False
    old_flag = False
    for year in test_name_dict_list[name]:
        year = int(year)
        if year >= 1 and year <= 5:
            old_flag = True
        if year >= 6 and year <= 10:
            new_flag = True
    if new_flag and old_flag:
        always_name_list.append(name)
    elif new_flag:
        new_name_list.append(name)
    elif old_flag:
        old_name_list.append(name)
    else:
        print(name,'???')
        
print('len new_name_list',len(new_name_list),'len old_name_list',len(old_name_list))
print('len always_name_list',len(always_name_list))

Yang Kun ???
 Sheirly ???
Chia Liang ???
Tai Yoong ???
Chee Guang ???
Suit Yan ???
Ming Kheong ???
Yew Fei ???
Chia Dong ???
Boon Xui ???
Xiao Yie ???
Yan Fang ???
Su Fen ???
Xue Zhe ???
Zi Kuang ???
Peng Wei ???
Poh Keong ???
Chen Ghee ???
Zhin Fong ???
Xing Hong ???
Hoy Leong ???
Kun Yong ???
Cheak Soon ???
Yan Xian ???
Kok Hieng ???
Qin Zhi ???
Ngee Yong ???
Ding Shen ???
Chin Lam ???
Chong Woei ???
Shuoh Jer ???
Yong Quan ???
Kae Chyi ???
Teck Jing ???
Lim Neo ???
Khai Yeong ???
Yi Rei ???
 Jesllen ???
Ling Ching ???
Jinq Yee ???
Shan Yu ???
Chong You ???
Kit Shan ???
Chee Shin ???
Kwang Hui ???
Fang Ni ???
Jia Peng ???
Wen Quan ???
Yang Feng ???
Woei Seng ???
Kian Aik ???
Weai Ming ???
Khoi Khei ???
Yien Haw ???
Fu Kai ???
Wei Xin ???
Shin Lin ???
 Yithui ???
Keh Xin ???
Pang Wei ???
Jin Hon ???
Sheue Hui ???
Yi Xue ???
Surgoh Jos ???
Bon Sheng ???
Kim May ???
Zhou Yee ???
 Gavin ???
Pei Kean ???
Jian Huan ???
Chee Vee ???
Chyi Yee ???
Joon Kit ???
Khaa Wei ???
Ai Siang ???
Ban Hi

Kia Sheng ???
Yungshi Chelsey ???
Hon Boon ???
Jin Zhi ???
Zheng Chen ???
Bei En ???
Suh Fang ???
Meng Quan ???
Yishin Denise ???
Zhi Ting ???
Mei Xin ???
Kian Woon ???
Sie Ying ???
Chew Shien ???
Wen Fhu ???
Hao Jun ???
Chen Leng ???
Pey Sin ???
Shir Leen ???
Xi Zhen ???
Lye Ping ???
Xi En ???
Ruoh Lynn ???
Leou Chyn ???
Ting Jia ???
Bing Cun ???
Sing Nee ???
Bing Fong ???
Jia Xuan ???
Ching Ann ???
Sheng Liang ???
Heng Hun ???
Fan Jing ???
Wen Loong ???
Mei Fui ???
Wan Seng ???
Shiau Zia ???
Zheng Yu ???
Le Yang ???
Hao Wee ???
Suh Chyi ???
Dick Xiang ???
Wey Bing ???
Ai Qi ???
Siew Min ???
Zhi Fu ???
Ying Tong ???
Khar Zhin ???
Yong Chuen ???
Bao Ning ???
Sin Yuan ???
Wen Bin ???
Xing Tian ???
Feng Yi ???
Chiu Min ???
Pey Tyng ???
Poh Thian ???
Chin Fang ???
Whye Khan ???
Kuan Ming ???
Vern Sing ???
Yok Min ???
 Zheyi ???
 Ching ???
Hee Ping ???
Kang Mei ???
Sze Hance ???
Yu En ???
Lu Juan ???
Ching Lam ???
Xue Ching ???
Xin Syan ???
Cau Yen ???
Lien Jing ???
Cheau May ???
Ser Thin 

In [15]:
test_name_dict_list

{'Han Sen': {7: 1},
 'Yoke Foong': {7: 2, 9: 1},
 'Soo San': {7: 1},
 'Kian Sin': {7: 2, 10: 1},
 'Khan Thai': {7: 1},
 'Fuk Chuan': {7: 1},
 'Wye Keong': {7: 1},
 'Kah Cheng': {7: 1},
 'Tak Fei': {7: 1},
 'Huo Chong': {7: 1, 6: 1},
 'Yong Nam': {7: 1, 9: 1},
 'Yuek Mee': {7: 1},
 'Wee Hong': {7: 1, 8: 3, 9: 1, 10: 2, 11: 2},
 'Kok Leong': {7: 2, 8: 3, 9: 3, 10: 2},
 'Yew Shin': {7: 1},
 'Chin Gang': {7: 1},
 'Yean Sun': {7: 2},
 'Tze Ming': {7: 1, 8: 4, 9: 2},
 'Kam Hong': {7: 2, 9: 2},
 'Ming Khiang': {7: 1},
 'Fang Hong': {7: 1},
 'Choon Chong': {7: 1, 8: 1, 9: 1},
 'Ee Choon': {7: 1, 8: 1},
 'Chong Tat': {7: 1, 8: 2},
 'Yen Hau': {7: 1},
 'Por Jin': {7: 1},
 'Kwong Soon': {7: 1},
 'Chee Kin': {7: 1, 8: 2, 9: 5, 11: 1},
 'Lay Cheng': {7: 1, 8: 2, 10: 1},
 'Shan Yong': {7: 1},
 'Kun Hao': {7: 1},
 'Say Guan': {7: 1},
 ' Aun': {7: 1},
 'Yen Fong': {7: 1, 8: 1, 9: 1},
 'Heng Chin': {7: 1},
 'Keng Siang': {7: 1},
 'Ling King': {7: 2},
 'Eng Hock': {7: 1, 8: 3},
 'Mei Lin': {7: 1, 8: 4, 

In [32]:
# filename = './data_output/test_name_category_list'
# pickle.dump(test_name_dict_list,open(filename, 'wb'))

# filename = './data_output/test_name_year_list'
# pickle.dump(test_name_year_dict_list,open(filename, 'wb'))

In [16]:
with open('./pkl_data/test_name_category_list', 'rb') as handle:
    test_name_dict_list = pickle.loads(handle.read())
    
with open('./pkl_data/test_name_year_list', 'rb') as handle:
    test_name_year_dict_list = pickle.loads(handle.read())

# Check Result

In [43]:
# FN1

x_feature = ['FN1']
y_feature = ['BirthYear']

result = age_prediction_FN1.predict(msadata[x_feature])
prob = age_prediction_FN1.predict_proba(msadata[x_feature])

count_average_error(msadata,result)

# Save the result and model
filename = './concate_age_model_result/age_predicton_FN1'
pickle.dump(result, open(filename, 'wb'))
print("Result save successfully!")

filename = './concate_age_model_prob/age_predicton_FN1'
pickle.dump(prob, open(filename, 'wb'))
print("Prob save successfully!")

Total test list: 30565
Accuracy: 0.1288728938328153
Mean Year Absolute Error: 15.700147227220677
Result save successfully!
Prob save successfully!


In [44]:
# FN2

x_feature = ['FN2']
y_feature = ['BirthYear']

result = age_prediction_FN2.predict(msadata[x_feature])
prob = age_prediction_FN2.predict_proba(msadata[x_feature])

count_average_error(msadata,result)

# Save the result and model
filename = './concate_age_model_result/age_prediction_FN2'
pickle.dump(result, open(filename, 'wb'))
print("Result save successfully!")

filename = './concate_age_model_prob/age_prediction_FN2'
pickle.dump(prob, open(filename, 'wb'))
print("Prob save successfully!")

Total test list: 30565
Accuracy: 0.019859316211352854
Mean Year Absolute Error: 10.652053001799445
Result save successfully!
Prob save successfully!


In [45]:
# FN12

x_feature = ['FN1','FN2']
y_feature = ['BirthYear']

result = age_prediction_FN12.predict(msadata[x_feature])
prob = age_prediction_FN12.predict_proba(msadata[x_feature])

count_average_error(msadata,result)

# Save the result and model
filename = './concate_age_model_result/age_prediction_FN12'
pickle.dump(result, open(filename, 'wb'))
print("Result save successfully!")

filename = './concate_age_model_prob/age_prediction_FN12'
pickle.dump(prob, open(filename, 'wb'))
print("Prob save successfully!")

Total test list: 30565
Accuracy: 0.2864387371176182
Mean Year Absolute Error: 4.5822345820382795
Result save successfully!
Prob save successfully!


In [46]:
# sonin

x_feature = ['FN1_sonin_index','FN2_sonin_index']
y_feature = ['BirthYear']

result = age_prediction_sonin.predict(msadata[x_feature])
prob = age_prediction_sonin.predict_proba(msadata[x_feature])

count_average_error(msadata,result)

# Save the result and model
filename = './concate_age_model_result/age_prediction_sonin'
pickle.dump(result, open(filename, 'wb'))
print("Result save successfully!")

filename = './concate_age_model_prob/age_prediction_sonin'
pickle.dump(prob, open(filename, 'wb'))
print("Prob save successfully!")

Total test list: 30565
Accuracy: 0.18328153116309504
Mean Year Absolute Error: 11.183739571405201
Result save successfully!
Prob save successfully!


In [47]:
# muin

x_feature = ['FN1_muin_index','FN2_muin_index']
y_feature = ['BirthYear']

result = age_prediction_muin.predict(msadata[x_feature])
prob = age_prediction_muin.predict_proba(msadata[x_feature])

count_average_error(msadata,result)

# Save the result and model
filename = './concate_age_model_result/age_prediction_muin'
pickle.dump(result, open(filename, 'wb'))
print("Result save successfully!")

filename = './concate_age_model_prob/age_prediction_muin'
pickle.dump(prob, open(filename, 'wb'))
print("Prob save successfully!")

Total test list: 30565
Accuracy: 0.14745624079829872
Mean Year Absolute Error: 16.36077212497955
Result save successfully!
Prob save successfully!


In [48]:
# soninmuin

x_feature = ['FN1_sonin_index','FN2_sonin_index','FN1_muin_index','FN2_muin_index']
y_feature = ['BirthYear']

result = age_prediction_soninmuin.predict(msadata[x_feature])
prob = age_prediction_soninmuin.predict_proba(msadata[x_feature])

count_average_error(msadata,result)

# Save the result and model
filename = './concate_age_model_result/age_prediction_soninmuin'
pickle.dump(result, open(filename, 'wb'))
print("Result save successfully!")

filename = './concate_age_model_prob/age_prediction_soninmuin'
pickle.dump(prob, open(filename, 'wb'))
print("Prob save successfully!")

Total test list: 30565
Accuracy: 0.14336659577948635
Mean Year Absolute Error: 14.7213479469982
Result save successfully!
Prob save successfully!


In [49]:
# FN1sonin1

x_feature = ['FN1','FN1_sonin_index']
y_feature = ['BirthYear']

result = age_predicton_FN1sonin1.predict(msadata[x_feature])
prob = age_predicton_FN1sonin1.predict_proba(msadata[x_feature])

count_average_error(msadata,result)

# Save the result and model
filename = './concate_age_model_result/age_predicton_FN1sonin1'
pickle.dump(result, open(filename, 'wb'))
print("Result save successfully!")

filename = './concate_age_model_prob/age_predicton_FN1sonin1'
pickle.dump(prob, open(filename, 'wb'))
print("Prob save successfully!")

Total test list: 30565
Accuracy: 0.1785048257811222
Mean Year Absolute Error: 12.630721413381318
Result save successfully!
Prob save successfully!


In [50]:
# FN1muin1

x_feature = ['FN1','FN1_muin_index']
y_feature = ['BirthYear']

result = age_predicton_FN1muin1.predict(msadata[x_feature])
prob = age_predicton_FN1muin1.predict_proba(msadata[x_feature])

count_average_error(msadata,result)

# Save the result and model
filename = './concate_age_model_result/age_predicton_FN1muin1'
pickle.dump(result, open(filename, 'wb'))
print("Result save successfully!")

filename = './concate_age_model_prob/age_predicton_FN1muin1'
pickle.dump(prob, open(filename, 'wb'))
print("Prob save successfully!")

Total test list: 30565
Accuracy: 0.1754294127269753
Mean Year Absolute Error: 12.700768853263536
Result save successfully!
Prob save successfully!


In [51]:
# FN2sonin2

x_feature = ['FN2','FN2_sonin_index']
y_feature = ['BirthYear']

result = age_predicton_FN2sonin2.predict(msadata[x_feature])
prob = age_predicton_FN2sonin2.predict_proba(msadata[x_feature])

count_average_error(msadata,result)

# Save the result and model
filename = './concate_age_model_result/age_predicton_FN2sonin2'
pickle.dump(result, open(filename, 'wb'))
print("Result save successfully!")

filename = './concate_age_model_prob/age_predicton_FN2sonin2'
pickle.dump(prob, open(filename, 'wb'))
print("Prob save successfully!")

Total test list: 30565
Accuracy: 0.29874038933420577
Mean Year Absolute Error: 5.303320791755276
Result save successfully!
Prob save successfully!


In [52]:
# FN2muin2

x_feature = ['FN2','FN2_muin_index']
y_feature = ['BirthYear']

result = age_predicton_FN2muin2.predict(msadata[x_feature])
prob = age_predicton_FN2muin2.predict_proba(msadata[x_feature])

count_average_error(msadata,result)

# Save the result and model
filename = './concate_age_model_result/age_predicton_FN2muin2'
pickle.dump(result, open(filename, 'wb'))
print("Result save successfully!")

filename = './concate_age_model_prob/age_predicton_FN2muin2'
pickle.dump(prob, open(filename, 'wb'))
print("Prob save successfully!")

Total test list: 30565
Accuracy: 0.27286111565516113
Mean Year Absolute Error: 6.639522329461803
Result save successfully!
Prob save successfully!


In [53]:
# FNsonin

x_feature = ['FN1','FN2','FN1_sonin_index','FN2_sonin_index']
y_feature = ['BirthYear']

result = age_prediction_FNsonin.predict(msadata[x_feature])
prob = age_prediction_FNsonin.predict_proba(msadata[x_feature])

count_average_error(msadata,result)

# Save the result and model
filename = './concate_age_model_result/age_prediction_FNsonin'
pickle.dump(result, open(filename, 'wb'))
print("Result save successfully!")

filename = './concate_age_model_prob/age_prediction_FNsonin'
pickle.dump(prob, open(filename, 'wb'))
print("Prob save successfully!")

Total test list: 30565
Accuracy: 0.26383117945362344
Mean Year Absolute Error: 8.399738262718795
Result save successfully!
Prob save successfully!


In [54]:
# FNmuin

x_feature = ['FN1','FN2','FN1_muin_index','FN2_muin_index']
y_feature = ['BirthYear']

result = age_prediction_FNmuin.predict(msadata[x_feature])
prob = age_prediction_FNmuin.predict_proba(msadata[x_feature])

count_average_error(msadata,result)

# Save the result and model
filename = './concate_age_model_result/age_prediction_FNmuin'
pickle.dump(result, open(filename, 'wb'))
print("Result save successfully!")

filename = './concate_age_model_prob/age_prediction_FNmuin'
pickle.dump(prob, open(filename, 'wb'))
print("Prob save successfully!")

Total test list: 30565
Accuracy: 0.23304433175200392
Mean Year Absolute Error: 8.886962211680027
Result save successfully!
Prob save successfully!


In [55]:
# 6F

x_feature = ['FN1','FN2','FN1_sonin_index','FN2_sonin_index','FN1_muin_index','FN2_muin_index']
y_feature = ['BirthYear']

result = age_prediction_6F.predict(msadata[x_feature])
prob = age_prediction_6F.predict_proba(msadata[x_feature])

count_average_error(msadata,result)

# Save the result and model
filename = './concate_age_model_result/age_prediction_6F'
pickle.dump(result, open(filename, 'wb'))
print("Result save successfully!")

filename = './concate_age_model_prob/age_prediction_6F'
pickle.dump(prob, open(filename, 'wb'))
print("Prob save successfully!")

Total test list: 30565
Accuracy: 0.2495664976280059
Mean Year Absolute Error: 9.914837232128251
Result save successfully!
Prob save successfully!


In [56]:
# Only FN1 features

x_feature = ['FN1','FN1_sonin_index','FN1_muin_index']
y_feature = ['BirthYear']

result = age_predicton_FN1Features.predict(msadata[x_feature])
prob = age_predicton_FN1Features.predict_proba(msadata[x_feature])

count_average_error(msadata,result)

# Save the result and model
filename = './concate_age_model_result/age_predicton_FN1Features'
pickle.dump(result, open(filename, 'wb'))
print("Result save successfully!")

filename = './concate_age_model_prob/age_predicton_FN1Features'
pickle.dump(prob, open(filename, 'wb'))
print("Prob save successfully!")

Total test list: 30565
Accuracy: 0.21426468182561753
Mean Year Absolute Error: 10.224472435792574
Result save successfully!
Prob save successfully!


In [57]:
# Only FN2 features

x_feature = ['FN2','FN2_sonin_index','FN2_muin_index']
y_feature = ['BirthYear']

result = age_predicton_FN2Features.predict(msadata[x_feature])
prob = age_predicton_FN2Features.predict_proba(msadata[x_feature])

count_average_error(msadata,result)

# Save the result and model
filename = './concate_age_model_result/age_predicton_FN2Features'
pickle.dump(result, open(filename, 'wb'))
print("Result save successfully!")

filename = './concate_age_model_prob/age_predicton_FN2Features'
pickle.dump(prob, open(filename, 'wb'))
print("Prob save successfully!")

Total test list: 30565
Accuracy: 0.3354163258629151
Mean Year Absolute Error: 5.77114346474726
Result save successfully!
Prob save successfully!


# W2V

In [17]:
#使用word to vector
from gensim.models import word2vec

with open('../TaiwaneseDataset/data_output/graduate/files/pinyin_wordvec_list', 'rb') as handle:
    moe_model = pickle.load(handle)

In [18]:
%%time 

def add_word_vector(word,n):
    if word in moe_model:
        return moe_model[word][n]
    else:
        return 0
    
w_feature = []
x_feature = []
FirstName1_list = msadata.FirstName1.str.lower().tolist()
FirstName2_list = msadata.FirstName2.str.lower().tolist()

for i in range(0,100):
    tmp1_list = []
    tmp2_list = []
    for word in FirstName1_list:
        tmp1_list.append(add_word_vector(word,i))
    for word in FirstName2_list:
        tmp2_list.append(add_word_vector(word,i))

    msadata['FN1_wv_' + str(i)] = tmp1_list
    msadata['FN2_wv_' + str(i)] = tmp2_list
    
    x_feature.append('FN1_wv_' + str(i))
    x_feature.append('FN2_wv_' + str(i))
    w_feature.append('FN1_wv_' + str(i))
    w_feature.append('FN2_wv_' + str(i))

%xdel tmp1_list
%xdel tmp2_list

CPU times: user 2.35 s, sys: 37.6 ms, total: 2.39 s
Wall time: 2.39 s


In [60]:
# SAVE DATA INTO CSV FILE
msadata.to_csv('MY_TESTDATA_W2V.csv', index = False)

In [19]:
# READ MY DATA INTO CSV FILE
msadata = pd.read_csv('MY_TESTDATA_W2V.csv', parse_dates = True)

/home/yvonnechow/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning:

Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.



In [74]:
%%time

# W2V

x_f = ['w2v']

x_feature = get_x_feature(x_f)
y_feature = ['BirthYear']
    
age_test = basic_RFClassifier(reconcatedata, x_feature, y_feature, 10)

# Save the model to disk
filename = './concate_age_model/age_predicton_w2v.sav'
pickle.dump(age_test, open(filename, 'wb'))

print("Model save successfully!")

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.5536617310808711
Test Accuracy  ::  0.2116129700611209


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.5549448353941439
Test Accuracy  ::  0.2098829379467523


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.5542122671876041
Test Accuracy  ::  0.20940640215477055


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.5557262414811197
Test Accuracy  ::  0.21003833005283332


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.5550247519257664
Test Accuracy  ::  0.20987257847301358
Average Train Accuracy ::  0.5546895464736831
Average Test Accuracy  ::  0.2100828757899099
Model save successfully!
CPU times: user 1h 30min 40s, sys: 9.02 s, total: 1h 30min 49s
Wall time: 6min 10s


In [75]:
%%time

# W2V+6F

x_f = ['w2v']

x_feature = get_x_feature(x_f)
x_feature = x_feature + ['FN1','FN2','FN1_muin_index','FN2_muin_index','FN1_sonin_index','FN2_sonin_index']
y_feature = ['BirthYear']
    
age_test = basic_RFClassifier(reconcatedata, x_feature, y_feature, 10)

# Save the model to disk
filename = './concate_age_model/age_predicton_w2v6F.sav'
pickle.dump(age_test, open(filename, 'wb'))
print("Model save successfully!")

/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 1
Train Accuracy ::  0.557138100206451
Test Accuracy  ::  0.2107324147933285


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 2
Train Accuracy ::  0.5557972783981175
Test Accuracy  ::  0.2110949963741842


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



...

Run times: 8
Train Accuracy ::  0.5567296379337137
Test Accuracy  ::  0.20954107531337407


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 9
Train Accuracy ::  0.55717361866495
Test Accuracy  ::  0.20965502952450016


/home/yvonnechow/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Run times: 10
Train Accuracy ::  0.5557395609030568
Test Accuracy  ::  0.2140474463897234
Average Train Accuracy ::  0.5567802517370747
Average Test Accuracy  ::  0.21084326116233293
Model save successfully!
CPU times: user 1h 28min 18s, sys: 7.24 s, total: 1h 28min 25s
Wall time: 6min 12s


# Read W2V Model

In [76]:
# Read model file

filename = './concate_age_model/age_predicton_w2v.sav'
age_predicton_w2v = pickle.load(open(filename, 'rb'))

filename = './concate_age_model/age_predicton_w2v6F.sav'
age_predicton_w2v6F = pickle.load(open(filename, 'rb'))

# Test W2V Result

In [77]:
%%time
# Only character
x_f = ['w2v']

y_feature = 'BirthYear'
x_feature = get_x_feature(x_f)

result = age_predicton_w2v.predict(msadata[x_feature])
prob = age_predicton_w2v.predict_proba(msadata[x_feature])

count_average_error(msadata,result)

Total test list: 30565
Accuracy: 0.13643055782758057
Mean Year Absolute Error: 18.6885653525274
CPU times: user 6.35 s, sys: 56.3 ms, total: 6.4 s
Wall time: 2.32 s


In [78]:
%%time
# Only character+6F
x_f = ['w2v']

y_feature = 'BirthYear'
x_feature = get_x_feature(x_f)
x_feature = x_feature + ['FN1','FN2','FN1_muin_index','FN2_muin_index','FN1_sonin_index','FN2_sonin_index']

result = age_predicton_w2v6F.predict(msadata[x_feature])
prob = age_predicton_w2v6F.predict_proba(msadata[x_feature])

count_average_error(msadata,result)

Total test list: 30565
Accuracy: 0.2185833469654834
Mean Year Absolute Error: 11.544315393423851
CPU times: user 5.74 s, sys: 64.2 ms, total: 5.81 s
Wall time: 2.18 s
